## Проект спринт 12
**Название проекта**:  
Цифра     
_____
**Описание исследования**:  
Построение модели предсказания коэффициента восстановления золота из золотосодержащей руды на основании данных с параметрами добычи и очистки.  
_____
**Цель исследования**:  
Оптимизация производства, чтобы не запускать предприятие с убыточными характеристиками.
_____
**План исследования.**  
[Шаг 1. Загрузка и подготовка данных](#step1)  
[Шаг 2. Анализ данных](#step2)  
[Шаг 3. Построение модели](#step3)  
_____
**Исходные данные.**
Исходные файлы, содержащие данные геологоразведки трёх регионов:  
- [gold_recovery_full_new.csv](gold_recovery_full_new.csv) - полные исходные данные
- [gold_recovery_test_new.csv](gold_recovery_test_new.csv) - валидационная выборка
- [gold_recovery_train_new.csv](gold_recovery_train_new.csv) - обучающая выборка   


1. Подготовьте данные
1.1. Откройте файлы и изучите их.
Путь к файлам:
/datasets/gold_recovery_train_new.csv. Скачать датасет
/datasets/gold_recovery_test_new.csv. Скачать датасет
/datasets/gold_recovery_full_new.csv. Скачать датасет
1.2. Проверьте, что эффективность обогащения рассчитана правильно. Вычислите её на обучающей выборке для признака rougher.output.recovery. Найдите MAE между вашими расчётами и значением признака. Опишите выводы.
1.3. Проанализируйте признаки, недоступные в тестовой выборке. Что это за параметры? К какому типу относятся?
1.4. Проведите предобработку данных.
2. Проанализируйте данные
2.1. Посмотрите, как меняется концентрация металлов (Au, Ag, Pb) на различных этапах очистки. Опишите выводы.
2.2.  Сравните распределения размеров гранул сырья на обучающей и тестовой выборках. Если распределения сильно отличаются друг от друга, оценка модели будет неправильной.
2.3. Исследуйте суммарную концентрацию всех веществ на разных стадиях: в сырье, в черновом и финальном концентратах. 
3. Постройте модель
3.1. Напишите функцию для вычисления итоговой sMAPE.
3.2. Обучите разные модели и оцените их качество кросс-валидацией. Выберите лучшую модель и проверьте её на тестовой выборке. Опишите выводы.

In [1]:
import pandas as pd

In [6]:
full = pd.read_csv('gold_recovery_full_new.csv')
test = pd.read_csv('gold_recovery_test_new.csv')
train = pd.read_csv('gold_recovery_train_new.csv')
test.shape

(5290, 53)

In [19]:
def loc_columns(df, columns):
    return df.loc[:,df.columns.str.startswith(columns)]

In [22]:
test_rougher = loc_columns(test, 'rougher.input')
display(test_rougher.sample(10))

,rougher.input.feed_ag,rougher.input.feed_pb,rougher.input.feed_rate,rougher.input.feed_size,rougher.input.feed_sol,rougher.input.feed_au,rougher.input.floatbank10_sulfate,rougher.input.floatbank10_xanthate,rougher.input.floatbank11_sulfate,rougher.input.floatbank11_xanthate
2875,5.408221,2.859734,548.251273,55.854757,41.112380,6.747391,17.000441,5.451210,16.998454,5.452988
3410,7.006176,2.605347,605.691686,61.635119,42.946925,6.371920,17.996637,5.598859,17.998381,5.600755
3935,9.814181,5.256943,583.058163,54.128592,40.272455,11.150697,16.998245,6.701929,16.997527,6.690441
4068,9.920381,4.675297,684.620241,112.383021,43.389877,10.168976,18.026623,6.846946,18.019143,6.839854
936,9.955726,4.207072,360.026909,41.884007,32.052343,11.282187,12.997893,6.542264,13.014651,6.527279
436,9.042340,3.224999,359.887928,45.646367,36.368390,8.760443,8.004699,6.294974,8.000800,6.299309
3205,9.600282,3.953926,487.528566,53.459277,41.266239,10.228950,17.006187,6.299361,17.009925,6.299031
2929,7.424976,3.150998,587.308031,58.349200,42.589012,7.866920,17.001248,7.336973,17.001404,7.332487
2062,10.431271,4.653539,370.286219,43.259144,33.623740,10.259591,10.499138,6.400916,10.503317,6.400020
578,9.828523,3.097134,450.041494,43.583889,34.955607,9.008218,9.506002,6.584645,9.527417,6.599652


In [ ]:
test_rougher['recovery_calc'] = 1
# C (F - T)
# F (C - T)
# C - Au after
# F - Au before
# T - Au in tails

In [23]:
test['recovery_f'] = test['rougher.input.feed_au'] / (test['rougher.input.feed_ag'] + test['rougher.input.feed_pb'] + test['rougher.input.feed_sol'] + test['rougher.input.feed_au'])